In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import numpy as np
from scipy.stats import norm, foldnorm
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv("dew-point-pitching/data.csv")
df['pk'] = df['PITCHER_KEY'].apply(lambda x: str(x))
df

,PID,INNING_KEY,BATTER_IN_INNING_KEY,PITCH_NUMBER,OUT_KEY,BALLS,STRIKES,IS_RUNNER_ON_1B,IS_RUNNER_ON_2B,IS_RUNNER_ON_3B,...,SPIN_RATE_ABSOLUTE,RELEASE_SPEED,RELEASE_SIDE,RELEASE_HEIGHT,RELEASE_EXTENSION,HORIZONTAL_APPROACH_ANGLE,VERTICAL_APPROACH_ANGLE,PLATE_X,PLATE_Z,pk
0,197,1,1,1,0,0,0,0,0,0,...,2402.459961,100.764999,-2.42717,6.08550,6.46,1.612990,-5.76003,0.197045,1.91282,668881
1,348,1,2,1,1,0,0,0,0,0,...,2273.459961,100.431000,-2.52253,5.97044,6.56,0.631016,-4.60990,-0.943079,2.87744,668881
2,1072,1,2,2,1,1,0,0,0,0,...,2314.459961,89.848999,-2.64242,5.89410,6.71,3.330480,-6.76551,0.012025,2.16833,668881
3,420,1,3,1,1,0,0,1,0,0,...,2340.459961,89.810997,-2.52429,5.90717,6.55,3.804230,-6.87764,0.584736,2.11319,668881
4,198,1,3,2,1,0,1,1,0,0,...,2445.459961,89.014397,-2.68096,6.10905,6.37,2.385210,-5.89879,-0.634335,3.45222,668881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9884,22663,9,4,2,2,1,0,1,0,0,...,2139.620117,92.623001,-2.01709,6.18892,5.81,2.164890,-4.58267,0.750023,3.40322,643361
9885,28642,9,4,3,2,1,1,1,0,0,...,2104.620117,88.973701,-2.22191,6.09663,5.71,3.371970,-5.39125,0.872825,3.17087,643361
9886,22664,9,4,4,2,2,1,1,0,0,...,2142.620117,92.265297,-1.99639,6.21051,5.83,0.063614,-5.65863,-0.988207,2.56097,643361
9887,22665,9,4,5,2,2,2,1,0,0,...,1968.619995,84.541603,-2.25819,6.13594,5.60,0.865016,-6.89677,-0.278290,2.14642,643361


In [25]:
cols = ['THROW_SIDE_KEY', 'PITCH_TYPE_TRACKED_KEY', 'INDUCED_VERTICAL_BREAK', 'HORIZONTAL_BREAK', 'SPIN_RATE_ABSOLUTE', 'RELEASE_SPEED', 'RELEASE_SIDE', 'RELEASE_HEIGHT', 'RELEASE_EXTENSION', 'HORIZONTAL_APPROACH_ANGLE', 'VERTICAL_APPROACH_ANGLE', 'PLATE_X', 'PLATE_Z']
data_cols = ['INDUCED_VERTICAL_BREAK', 'HORIZONTAL_BREAK', 'SPIN_RATE_ABSOLUTE', 'RELEASE_SPEED', 'RELEASE_SIDE', 'RELEASE_HEIGHT', 'RELEASE_EXTENSION', 'HORIZONTAL_APPROACH_ANGLE', 'VERTICAL_APPROACH_ANGLE', 'PLATE_X', 'PLATE_Z']
data_cols_w_inning = ['INNING_KEY', 'INDUCED_VERTICAL_BREAK', 'HORIZONTAL_BREAK', 'SPIN_RATE_ABSOLUTE', 'RELEASE_SPEED', 'RELEASE_SIDE', 'RELEASE_HEIGHT', 'RELEASE_EXTENSION', 'HORIZONTAL_APPROACH_ANGLE', 'VERTICAL_APPROACH_ANGLE', 'PLATE_X', 'PLATE_Z']


In [4]:
pitchers = df['PITCHER_KEY'].unique()
pitchers

array([668881, 518585, 622065, 571882, 571656, 664139, 666157, 665665,
       664747, 668933, 656818, 570666, 669270, 643361, 668984, 613564,
       571912, 686651, 608371, 596133, 594902, 611093, 641427, 674285,
       682227, 671096, 650960, 691094, 621219, 594580, 622088, 686730,
       592741, 664028, 592527, 683175, 680689])

In [60]:
# mask = (df['THROW_SIDE_KEY'] == 'R') & (df['PITCH_TYPE_TRACKED_KEY'] == 'SL')  & (df['PITCHER_KEY'] == 668881)
dfs = []
adj_r = []
for pitcher in pitchers:
    p_df = df[df['PITCHER_KEY'] == pitcher]
    pitches = [f'PITCH_TYPE_TRACKED_KEY_{i}' for i in p_df['PITCH_TYPE_TRACKED_KEY'].unique()][:-1]
    p_types = p_df['PITCH_TYPE_TRACKED_KEY'].unique()
    p_df = pd.get_dummies(p_df)

    # print(p_types)
    
    x = p_df[['HORIZONTAL_APPROACH_ANGLE', 'HORIZONTAL_BREAK'] + pitches]
    # x.corr().to_csv("p_correlations.csv")
    y = p_df['PLATE_X'].to_numpy()

    x = sm.add_constant(x)
    result = sm.OLS(y,x).fit()

    diff = y - np.array(result.predict(x))
    mu = np.mean(diff)
    std = np.std(diff)
    c = abs(mu) / std
    probs = foldnorm.cdf(abs(diff), c, mu, std) - foldnorm.cdf(0, c, mu, std)

    # p_ids = pd.DataFrame(p_df['PID'], columns=['PID'])
    adj_r.append(result.f_pvalue)

    # result.f_pvalue
#     p_ids['DEWPOINT_AFFECTED'] = probs

#     dfs.append(p_ids)

# dfs = pd.concat(dfs)
# dfs.to_csv('sub_files/pitch_lin_reg.csv', index=False)


In [61]:
np.mean(adj_r)

1.0331742166040383e-17